In [ ]:
# ML Algorithm

FUCK THAT... do a live-win probability model that takes in the score difference, number of seconds 
left in a game, and home/away PPP until that point

target variable is whether or not the team won (1/0)

then do a log reg problem
# Impute Whether or not team won via live_score in pbp dataset


# Outputs are live score graph, live score diff graph, and live win-prob graph

[SIMILAR TO MY THESIS MODEL]

# Data Viz Stuff

In [85]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

df = pd.read_csv('../data/interim/cleaned_pbp_data.csv')
try:
    df = df.drop('Unnamed: 0',axis=1)
except:
    pass

def add_team_id(df):
    team_id_df = pd.read_csv('../data/interim/team_id.csv')
    
    df = df.merge(team_id_df,how='left',on='Team_Name')
    
    return df

df.head()

,playByPlayId,game.awayTeam.fullName,game.awayTeam.awayTeamId,game.homeTeam.fullName,game.homeTeam.homeTeamId,defense.teamId,defense.name,offense.teamId,offense.name,game.gameId,game.date,clock,ato,eob,shot,sob,zone,possessionId,oPlayer.name,dPlayer.name,prPlayer.name,gameHalf,shotClock,homeEvent,turnover,offensiveString,secondaryString,ftAttempt,ftMade,Off_Lineup,Def_Lineup,Shot,Shooter,TOV,TOV_Player,PF,PF_Player,DREB,OREB,AST,STL,BLK,DREB_Player,OREB_Player,AST_Player,STL_Player,BLK_Player,FTA,FT_Made,FT_Miss,3PA,3P_Made,3P_Miss,2PA,2P_Made,2P_Miss,Open_Shot,High P&R,ISO,Transition,Post-Up,Spot-Up,Cut,Flash,seconds_left_in_game,seconds_into_game,Home_Team_LiveScore,Away_Team_LiveScore,Possession_Value
0,5c7b7ff685d07ba97689e591,Texas Tech Red Raiders,54457dd3300969b132fcfead,TCU Horned Frogs,54457dd3300969b132fcfea0,54457dd3300969b132fcfead,TexasTech,54457dd3300969b132fcfea0,TCU,5bd05bd0aeda71993902327e,2019-03-02T00:00:00.000Z,11820,False,True,406,False,False,5c7b7ff785d07ba97689e812,Kouat Noi,NaN,Kouat Noi,1,False,True,False,12 Kouat Noi > Cut > Screen > Miss 2 Pts,Shot > Kouat Noi > Any Type > 2 Point Attempt ...,0,0,"['Alex Robinson', 'Desmond Bane', 'JD Miller',...","['Davide Moretti', 'Jarrett Culver', 'Matt Moo...",1.0,Kouat Noi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,2382,0,0,0,0
1,5c7b7ff685d07ba97689e592,Texas Tech Red Raiders,54457dd3300969b132fcfead,TCU Horned Frogs,54457dd3300969b132fcfea0,54457dd3300969b132fcfead,TexasTech,54457dd3300969b132fcfea0,TCU,5bd05bd0aeda71993902327e,2019-03-02T00:00:00.000Z,11820,False,True,406,False,False,5c7b7ff785d07ba97689e812,Kouat Noi,NaN,Tariq Owens,1,False,True,False,12 Kouat Noi > Cut > Screen > Miss 2 Pts,Misc Stat > Defensive Rebound > Tariq Owens,0,0,"['Alex Robinson', 'Desmond Bane', 'JD Miller',...","['Davide Moretti', 'Jarrett Culver', 'Matt Moo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,2382,0,0,0,0
2,5c7b7ff685d07ba97689e594,Texas Tech Red Raiders,54457dd3300969b132fcfead,TCU Horned Frogs,54457dd3300969b132fcfea0,54457dd3300969b132fcfea0,TCU,54457dd3300969b132fcfead,TexasTech,5bd05bd0aeda71993902327e,2019-03-02T00:00:00.000Z,11530,True,False,0,False,False,5c7b7ff785d07ba97689e789,Davide Moretti,Alex Robinson,Alex Robinson,1,True,False,True,25 Davide Moretti > P&R Ball Handler > Left P&...,Misc Stat > Steal > Alex Robinson,0,0,"['Davide Moretti', 'Jarrett Culver', 'Matt Moo...","['Alex Robinson', 'Desmond Bane', 'JD Miller',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2353,29,0,0,0
3,5c7b7ff685d07ba97689e593,Texas Tech Red Raiders,54457dd3300969b132fcfead,TCU Horned Frogs,54457dd3300969b132fcfea0,54457dd3300969b132fcfea0,TCU,54457dd3300969b132fcfead,TexasTech,5bd05bd0aeda71993902327e,2019-03-02T00:00:00.000Z,11530,True,False,0,False,False,5c7b7ff785d07ba97689e789,Davide Moretti,Alex Robinson,Davide Moretti,1,True,False,True,25 Davide Moretti > P&R Ball Handler > Left P&...,Turnover > Any Type > Davide Moretti,0,0,"['Davide Moretti', 'Jarrett Culver', 'Matt Moo...","['Alex Robinson', 'Desmond Bane', 'JD Miller',...",NaN,NaN,1.0,Davide Moretti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2353,29,0,0,0
4,5c7b7ff685d07ba97689e598,Texas Tech Red Raiders,54457dd3300969b132fcfead,TCU Horned Frogs,54457dd3300969b132fcfea0,54457dd3300969b132fcfead,TexasTech,54457dd3300969b132fcfea0,TCU,5bd05bd0aeda71993902327e,2019-03-02T00:00:00.000Z,11340,False,False,430,False,False,5c7b7ff785d07ba97689e815,JD Miller,Davide Moretti,JD Miller,1,False,True,False,21 Kevin Samuel > Post-Up > Right Block > Defe...,Shot > JD Miller > Any Type > 2 Point Attempt ...,0,0,"['Alex Robinson', 'Desmond Bane', 'JD Miller',...","['Davide Moretti', 'Jarrett Culver', 'Matt

In [27]:
lineup_tm_df = df[['Off_Lineup','offense.name']].drop_duplicates(subset=['Off_Lineup'],keep='first')

In [29]:
# Lineup ORtg, DRtg, NetRtg, +/-

poss_df = df[['Off_Lineup','possessionId','Possession_Value']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Off_Lineup').count()

poss_df = poss_df.reset_index()
poss_df = poss_df.drop('Possession_Value',axis=1)
poss_df.columns = ['Off_Lineup','Offensive_Possessions']

def_poss_df = df[['Def_Lineup','possessionId','Possession_Value']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Def_Lineup').count()
def_poss_df = def_poss_df.reset_index()
def_poss_df = def_poss_df.drop('Possession_Value',axis=1)
def_poss_df.columns = ['Def_Lineup','Defensive_Possessions']

poss_df = poss_df.merge(def_poss_df,how='left',left_on='Off_Lineup',right_on='Def_Lineup')

poss_df = poss_df.drop('Def_Lineup',axis=1)

####################################
pts_df = df[['Off_Lineup','possessionId','Possession_Value']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Off_Lineup').sum()

pts_df = pts_df.reset_index()
pts_df.columns = ['Off_Lineup','Off_PTS_Scored']

def_pts_df = df[['Def_Lineup','possessionId','Possession_Value']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Def_Lineup').sum()

def_pts_df = def_pts_df.reset_index()
def_pts_df.columns = ['Def_Lineup','Def_PTS_Allowed']

pts_df = pts_df.merge(def_pts_df,how='left',left_on='Off_Lineup',right_on='Def_Lineup')
pts_df = pts_df.drop('Def_Lineup',axis=1)
####################################
tm_lineup_eff_df = poss_df.merge(pts_df,how='left',on='Off_Lineup')

tm_lineup_eff_df = tm_lineup_eff_df.rename(columns = {'Off_Lineup':'5-Man Lineup'})

tm_lineup_eff_df['ORtg'] = round(tm_lineup_eff_df['Off_PTS_Scored'] / tm_lineup_eff_df['Offensive_Possessions'],4)*100
tm_lineup_eff_df['DRtg'] = round(tm_lineup_eff_df['Def_PTS_Allowed'] / tm_lineup_eff_df['Defensive_Possessions'],4)*100
tm_lineup_eff_df['NetRtg'] = tm_lineup_eff_df['ORtg'] - tm_lineup_eff_df['DRtg']
tm_lineup_eff_df['+/-'] = tm_lineup_eff_df['Off_PTS_Scored'] - tm_lineup_eff_df['Def_PTS_Allowed']

tm_lineup_eff_df = tm_lineup_eff_df.merge(lineup_tm_df,how='left',left_on='5-Man Lineup',right_on='Off_Lineup')

tm_lineup_eff_df = tm_lineup_eff_df.drop('Off_Lineup',axis=1)
tm_lineup_eff_df = tm_lineup_eff_df.rename(columns = {'offense.name':'Team_Name'})

tm_lineup_eff_df = add_team_id(tm_lineup_eff_df)

tm_lineup_eff_df.to_csv('../data/processed/team_lineup_efficiency_data.csv')

In [35]:
tm_lineup_eff_df[tm_lineup_eff_df['Offensive_Possessions']>=50].sort_values(by='NetRtg',ascending=False)


,5-Man Lineup,Offensive_Possessions,Defensive_Possessions,Off_PTS_Scored,Def_PTS_Allowed,ORtg,DRtg,NetRtg,+/-,Team_Name,Team_ID
315,"['Brandone Francis', 'Davide Moretti', 'Jarret...",131,116.0,101,59.0,77.10,50.86,26.24,42.0,TexasTech,TXTC
224,"['Barry Brown', 'Dean Wade', 'Kamau Stokes', '...",331,303.0,235,147.0,71.00,48.51,22.49,88.0,KansasState,KAST
835,"['George Conditt IV', 'Lindell Wigginton', 'Ma...",56,62.0,50,42.0,89.29,67.74,21.55,8.0,IowaState,IAST
267,"['Brandon Knapper', 'Chase Harler', 'Derek Cul...",59,67.0,53,46.0,89.83,68.66,21.17,7.0,WestVirginia,WVU
345,"['Cameron Lard', 'Lindell Wigginton', 'Nick We...",61,63.0,46,35.0,75.41,55.56,19.85,11.0,IowaState,IAST
197,"['Barry Brown', 'Cartier Diarra', 'Dean Wade',...",95,96.0,78,60.0,82.11,62.50,19.61,18.0,KansasState,KAST
222,"['Barry Brown', 'Dean Wade', 'Kamau Stokes', '...",79,78.0,69,53.0,87.34,67.95,19.39,16.0,KansasState,KAST
644,"['Dedric Lawson', 'Devon Dotson', 'Mitch Light...",137,143.0,108,86.0,78.83,60.14,18.69,22.0,Kansas,KAN
340,"['Cameron Lard', 'Lindell Wigginton', 'Marial ...",180,234.0,142,141.0,78.89,60.26,18.63,1.0,IowaState,IAST
784,"['Dylan Osetkowski', 'Jase Febres', 'Jaxson Ha...",164,162.0,135,108.0,82.32,66.67,15.65,27.0,TexasAustin,TEX


In [97]:
# Indiv Player On/Off ORtg/Drtg splits
def generate_on_off_flag(df, player_name):
    df['KEEP']=False
    for idx,row in df.iterrows():
        if player_name in row['Off_Lineup']:
            df.at[idx,'KEEP']=True
        if player_name in row['Def_Lineup']:
            df.at[idx,'KEEP']=True
            
    try_df = df[df['KEEP']==True]
    try:
        ply_tm = try_df.reset_index()['offense.teamId'].iat[0]
    except:
        ply_tm = 'NONE'
    
    df = df[(df['game.homeTeam.homeTeamId'] == ply_tm) | (df['game.awayTeam.awayTeamId'] == ply_tm)]
    
    
    df = df.reset_index()
    df = df.drop('index',axis=1)

    df['Player_On_Off']=False
    df['Player_On_Def']=False
    df['Player_On']=False
    
    for idx, row in df.iterrows():
        if player_name in row['Off_Lineup']:
            df.at[idx,'Player_On_Off']=True
            df.at[idx,'Player_On']=True
        
        if player_name in row['Def_Lineup']:
            df.at[idx,'Player_On_Def'] = True
            df.at[idx,'Player_On']=True
    
    try:
        df = df.drop(['KEEP'],axis=1)
    except:
        pass
    
    return df, ply_tm


def generate_ply_on_off_split_df(df):
    # Auto-loop through all players in dataset
    my_list = []
    for item in list(set(df['Shooter'].to_list())):
        if str(item) == 'nan':
            pass
        else:
            my_list.append(item)
    
    final_df = pd.DataFrame(columns=["Player", "Team_ID", "Possessions","ORtg_Split","DRtg_Split","NetRtg_Split"])
    
    for player_search_name in my_list:
        print(player_search_name)
        
        ply_df, tm_id = generate_on_off_flag(df,player_search_name)

        g_df = ply_df[['Player_On','possessionId','Possession_Value']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Player_On').count()
        g_df = g_df.reset_index()
        g_df = g_df.drop('Possession_Value',axis=1)
        g_df.columns=['Player_On','Possessions']
        g_df['Possessions'] = g_df['Possessions']/2

        off_pts_df = ply_df[['Player_On','Possession_Value','possessionId']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Player_On').sum()
        off_pts_df = off_pts_df.reset_index()
        off_pts_df.columns = ['Player_On','PTS_Scored']

        def_pts_df = ply_df[['Player_On_Def','Possession_Value','possessionId']].sort_values(by='Possession_Value',ascending=False).drop_duplicates(subset='possessionId',keep='first').groupby('Player_On_Def').sum()
        def_pts_df = def_pts_df.reset_index()
        def_pts_df.columns = ['Player_On','PTS_Allowed']

        g_df = g_df.merge(off_pts_df,how='left',on='Player_On')
        g_df = g_df.merge(def_pts_df,how='left',on='Player_On')

        g_df['ORtg'] = round(g_df['PTS_Scored']/(g_df['Possessions']),4)*100
        g_df['DRtg'] = round(g_df['PTS_Allowed']/(g_df['Possessions']),4)*100
        g_df['NetRtg'] = g_df['ORtg'] - g_df['DRtg']
        g_df['+/-'] = g_df['PTS_Scored'] - g_df['PTS_Allowed']

        try:
            my_poss_count = g_df[g_df['Player_On']==True].reset_index()['Possessions'].iat[0]
            my_ortg_split = round(g_df[g_df['Player_On']==True].reset_index()['ORtg'].iat[0] - g_df[g_df['Player_On']==False].reset_index()['ORtg'].iat[0],3)
            my_drtg_split = round(g_df[g_df['Player_On']==True].reset_index()['DRtg'].iat[0] - g_df[g_df['Player_On']==False].reset_index()['DRtg'].iat[0],3)
            if my_drtg_split < -90:
                my_drtg_split = round(my_drtg_split/10,2)
                
            my_nrtg_split = round(g_df[g_df['Player_On']==True].reset_index()['NetRtg'].iat[0] - g_df[g_df['Player_On']==False].reset_index()['NetRtg'].iat[0],3)
        except:
            my_poss_count = 0
            my_ortg_split = 0
            my_drtg_split = 0
            my_nrtg_split = 0
            
        final_df.loc[len(final_df.index)] = [player_search_name, tm_id, my_poss_count, my_ortg_split, my_drtg_split, my_nrtg_split]

    return final_df

player_on_off_splits_df = generate_ply_on_off_split_df(df)

player_on_off_splits_df.head()

Deshawn Corprew
Terrence Lewis
Solomon Young
Jordan Mccabe
Charlie Moore
Tyrese Haliburton
Royce Hamm
Makol Mawien
Desmond Bane
Chris Teahan
Trey Doomes
Brady Manek
Emmitt Matthews Jr
Mitch Lightfoot
Kentrevious Jones
Quentin Grimes
Jase Febres
Jamuni Mcneace
Avery Benson
Levi  Stockard III
Michael Jacobson
Barry Brown
Norense Odiase
Andrew Gordon
Wesley Harris
Curtis Jones
Cameron Mcgriff
Flo  Thomba
Kendric Davis
Matthew Mayer
Logan Routt
Blake Nevins
Darius Allen
Freddie Gillespie
Devon Dotson
Michael Weathers
Austin Trice
Gabe Simpson
Kerwin Roach Jr.
Jarrett Culver
Esa Ahmad
Malik Ondigo
Dedric Lawson
Matt Coleman
Jermaine Haley
Jericho Sims
Alex Robinson
Luke  Major
Elijah Mitrou-Long
Russell Barlow
Hannes Polla
James Love III
Tariq  Owens
Nigel Shadd
Maurce Calloo
Josh Mballa
Tanner Taylor
Udoka Azubuike
Patrick Muldoon
Andrew Sorrells
Patrick Geha
Mike Mcguirl
Zoran Talley Jr
Miles Reynolds
Dean Wade
Makai Mason
George Conditt IV
Courtney Ramey
Tristan Clark
Cameron Lard
Parker

,Player,Team_ID,Possessions,ORtg_Split,DRtg_Split,NetRtg_Split
0,Deshawn Corprew,54457dd3300969b132fcfead,574.0,4.41,-10.65,110.92
1,Terrence Lewis,54457dd3300969b132fcfe95,73.0,6.81,-63.16,69.97
2,Solomon Young,54457dd3300969b132fcfe95,43.5,10.12,-65.27,75.39
3,Jordan Mccabe,54457dd3300969b132fcfeb4,947.5,10.03,-11.36,123.64
4,Charlie Moore,54457dd3300969b132fcfe9b,472.5,-7.08,-9.22,85.11


In [102]:
player_on_off_splits_df_2 = player_on_off_splits_df.merge(df[['offense.teamId','offense.name']].drop_duplicates(subset='offense.teamId',keep='first'),
                              how='left',left_on='Team_ID',right_on='offense.teamId')

In [103]:
player_on_off_splits_df_2 = player_on_off_splits_df_2.drop(['offense.teamId','Team_ID'],axis=1)
player_on_off_splits_df_2 = player_on_off_splits_df_2.rename(columns={'offense.name':'Team_Name'})
player_on_off_splits_df_2 = add_team_id(player_on_off_splits_df_2)

player_on_off_splits_df_2.head()

,Player,Possessions,ORtg_Split,DRtg_Split,NetRtg_Split,Team_Name,Team_ID
0,Deshawn Corprew,574.0,4.41,-10.65,110.92,TexasTech,TXTC
1,Terrence Lewis,73.0,6.81,-63.16,69.97,IowaState,IAST
2,Solomon Young,43.5,10.12,-65.27,75.39,IowaState,IAST
3,Jordan Mccabe,947.5,10.03,-11.36,123.64,WestVirginia,WVU
4,Charlie Moore,472.5,-7.08,-9.22,85.11,Kansas,KAN


In [104]:
player_on_off_splits_df_2.to_csv('../data/processed/player_eff_splits.csv')

In [ ]:
# Indiv Player PPP on different play-types


In [ ]:
# IF TIME --- SHOTQUALITY ALGO
